In [1]:
###Test to effectively remove the mixing evolution time from the protocol
###Since the precise step at which a dissipative channel is probabilitically sampled during the mixin time does not seem to matter in the quality of the spectrum
#we might as well "commute through" the dissipative channelt to the end of the T1 time scale or the beginning of the T2 time scale...
#and potentially eliminate the need to prpagate 
 
import openfermion as of

import sys
sys.path.append('./utils/')

from basis_utils import Sz,Sx,Sz, Sy
from basis_utils import read_spinach_info, build_list_ISTs, NormalizeBasis, build_symbolic_list_ISTs, MatRepLib
from simulation_utils import GenH0_Ham, HamMatRep, GenNOESYSpectrum, sqcosbell_2d_apod, GenFIDsignals
import scipy.io as spio
#from 
from scipy.linalg import expm
import cirq
import numpy as np
from matplotlib import pyplot as plt
from scipy.io import savemat
import pickle

from matplotlib.colors import TwoSlopeNorm



In [199]:
def Normalize_and_weightOps(ListOps):
    """
    ListOps contains the generators of evolution, either the representation of the coherent part of evolution or the jump operators 
    """
    List_weights=[]

    for i in range(len(ListOps)):
        List_weights.append(np.max(np.abs(np.linalg.eigvals(ListOps[i]))))

    #Normalize the operator according to the weights...
    Norm_ops =[]

    for i in range(len(ListOps)):
        Norm_ops.append(ListOps[i]/List_weights[i])

    return Norm_ops, List_weights


def QDriftChann(List_gens,time_evol):
    """
    Args:
    List_gens: list of the generators of the time evolution, it is assumed that the first element is H_{0}
    list_probs: list of the corresponding probabilities to sample the exponentials of generators
    Gamma: sum of the weights of each of the normalized operators
    time_evol: the time for which we perform the propagation
    """

    Norm_ops, List_weights = Normalize_and_weightOps(List_gens)

    List_weights = np.array(List_weights)
    Gamma =  np.sum(List_weights)

    pks = (1.0/Gamma)*List_weights

    Qdrift_chann = pks[0]*expm(-1j*Norm_ops[0]*time_evol*Gamma)
    for i in range(1,len(pks)):
        Qdrift_chann = pks[i]*expm(Norm_ops[i]*time_evol*Gamma)

    return Qdrift_chann


def BuildQDriftChann_fromOps(time_evol,Norm_ops,pks,Gamma):

    Qdrift_chann = pks[0]*expm(-1j*Norm_ops[0]*time_evol*Gamma)
    for i in range(1,len(pks)):
        Qdrift_chann = pks[i]*expm(Norm_ops[i]*time_evol*Gamma)

    return Qdrift_chann

def QDriftEvol_fromNormOps(Norm_ops,pks,Gamma,time_evol,Nsteps,rho):
    
    rho_c = np.copy(rho)

    if time_evol==0:
        chann = np.eye(rho_c.shape[0])
    else:
        chann= BuildQDriftChann_fromOps(time_evol/Nsteps,Norm_ops,pks,Gamma)

    for i in range(Nsteps):
        rho_c = chann@rho_c

    return rho_c



def QDriftEvol(List_gens,time_evol,Nsteps,rho):
    
    rho_c = np.copy(rho)

    if time_evol == 0:
        chann = np.eye(rho_c.shape[0])
    else:
        chann = QDriftChann(List_gens,time_evol/Nsteps)

    for i in range(Nsteps):
        rho_c = chann@rho_c

    return rho_c




def eff_QDriftChann_InitJump_Factorized(List_gens,time_evol,NSteps,U):
    """
    Calculation of an effective QDrift channel. Based on the observation that the action of jump operators are in essence rare events 
    """
    Norm_ops, List_weights = Normalize_and_weightOps(List_gens)

    List_weights = np.array(List_weights)
    Gamma =  np.sum(List_weights)

    pks = (1.0/Gamma)*List_weights

    Eff_probs = np.zeros(len(List_gens))

    Eff_probs[0] = pks[0]**(NSteps) #probability of sampling coherent evolution only

    ###We define this channel as the product of two channels: the right one applies the jump operator before the action of the unitary pulse U
    omega = time_evol*Gamma/NSteps
    Right_chan = pks[0]*expm(-1j*Norm_ops[0]*omega)@U
    for k in range(1,len(pks)):
        Right_chan+=NSteps*pks[k]*U@expm(omega*np.transpose(np.conjugate(U))@Norm_ops[k]@U)

    ##left channel...
    Left_chan = pks[0]**(NSteps-1)*expm(-1j*Norm_ops[0]*(NSteps-1)*omega)


    return Left_chan, Right_chan


def eff_QDriftChann_FinJump_Factorized(List_gens,time_evol,NSteps,U):
    """
    Calculation of an effective QDrift channel. Based on the observation that the action of jump operators are in essence rare events 
    """
    Norm_ops, List_weights = Normalize_and_weightOps(List_gens)

    List_weights = np.array(List_weights)
    Gamma =  np.sum(List_weights)

    pks = (1.0/Gamma)*List_weights

    Eff_probs = np.zeros(len(List_gens))

    Eff_probs[0] = pks[0]**(NSteps) #probability of sampling coherent evolution only

    ###We define this channel as the product of two channels: the right one applies the jump operator before the action of the unitary pulse U
    omega = time_evol*Gamma/NSteps
    Left_chan = pks[0]*U@expm(-1j*Norm_ops[0]*omega)
    for k in range(1,len(pks)):
        Left_chan+=NSteps*pks[k]*expm(omega*U@Norm_ops[k]@np.conjugate(np.transpose(U)))@U

    ##right channel...
    Right_chan = pks[0]**(NSteps-1)*expm(-1j*Norm_ops[0]*(NSteps-1)*omega)


    return Left_chan, Right_chan


def get_last_QDriftJump(List_gens,time_evol,NSteps,U):
    """
    Calculation of an effective QDrift channel. Based on the observation that the action of jump operators are in essence rare events 
    """
    Norm_ops, List_weights = Normalize_and_weightOps(List_gens)

    List_weights = np.array(List_weights)
    Gamma =  np.sum(List_weights)

    pks = (1.0/Gamma)*List_weights

    #Eff_probs = np.zeros(len(List_gens))

    #Eff_probs[0] = pks[0]**(NSteps) #probability of sampling coherent evolution only

    ###We define this channel as the product of two channels: the right one applies the jump operator before the action of the unitary pulse U
    omega = time_evol*Gamma/NSteps
    Left_chan = pks[0]*U@expm(-1j*Norm_ops[0]*omega)
    for k in range(1,len(pks)):
        Left_chan+=NSteps*pks[k]*expm(omega*U@Norm_ops[k]@np.conjugate(np.transpose(U)))@U

    ##right channel...
    #Right_chan = pks[0]**(NSteps-1)*expm(-1j*Norm_ops[0]*(NSteps-1)*omega)


    return pks[0]**(NSteps-1)*Left_chan



def eff_QDriftEndOfProtocol(List_gens,time_evol,NSteps,U):
    """
    Calculation of factors that compose a QDrift-type channel that probabilitically applies a Jump operator at the end of the NOESY protocol 
    """
    Norm_ops, List_weights = Normalize_and_weightOps(List_gens)

    List_weights = np.array(List_weights)
    Gamma =  np.sum(List_weights)

    pks = (1.0/Gamma)*List_weights

    #Eff_probs = np.zeros(len(List_gens))

    #Eff_probs[0] = pks[0]**(NSteps) #probability of sampling coherent evolution only

    ###We define this channel as the product of two channels: the right one applies the jump operator before the action of the unitary pulse U
    omega = time_evol*Gamma/NSteps
    #Left_chan = U@expm(-1j*Norm_ops[0]*omega)@np.conjugate(np.transpose(U))
    #Left_chan = pks[0]*U@expm(-1j*Norm_ops[0]*omega)@np.conjugate(np.transpose(U))
    #Left_chan = pks[0]*U@expm(-1j*Norm_ops[0]*omega)@np.conjugate(np.transpose(U))
    Left_chan = pks[0]*expm(-1j*Norm_ops[0]*omega)
#
    for k in range(1,len(pks)):
        Left_chan+=NSteps*pks[k]*expm(omega*U@Norm_ops[k]@np.conjugate(np.transpose(U)))

    ##right channel...
    #Right_chan = expm(-1j*Norm_ops[0]*(NSteps-1)*omega)

    Right_chan = pks[0]**(NSteps-1)*expm(-1j*Norm_ops[0]*(NSteps-1)*omega)


    return Left_chan, Right_chan



def GenFID_cohevol(Ham,T1,T2,rho0,coil,tmix,dt1,dt2,Lx,Ly):

    #R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham)*dt1)

    L_dt2 = expm((-1j*Ham)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    #print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    #print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    #print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    #print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    pulse_mix = expm(-1j*Ham*tmix)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    for i in range(Tpts1):
        
        rho_stack1_1.append(pulse_90y@pulse_mix@pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@pulse_mix@pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90y@pulse_mix@pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90y@pulse_mix@pulse_90my@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4


def GenFID_cohevol_noJsInTmix(Ham,HamNoJs,T1,T2,rho0,coil,tmix,dt1,dt2,Lx,Ly):

    #R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham)*dt1)

    L_dt2 = expm((-1j*Ham)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    #print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    #print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    #print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    #print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    pulse_mix = expm(-1j*HamNoJs*tmix)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    for i in range(Tpts1):
        
        rho_stack1_1.append(pulse_90y@pulse_mix@pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@pulse_mix@pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90y@pulse_mix@pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90y@pulse_mix@pulse_90my@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4


def GenFID_NoTmix_cohT1T2(Ham,List_jumps,T1,T2,rho0,coil,dt1,dt2,Lx,Ly):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    
    R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham)*dt1)

    L_dt2 = expm((-1j*Ham)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    #print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    #print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    for i in range(Tpts1):
        

        rho_stack1_1.append(pulse_90y@pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90y@pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90y@pulse_90my@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4


def GenFID_TmixInitJumpQFact_cohT1T2(Ham,List_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    
    R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham)*dt1)

    L_dt2 = expm((-1j*Ham)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    #print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    leftmix, rightmix = eff_QDriftChann_InitJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90x)
    pulse_mix1 = leftmix@rightmix
    leftmix, rightmix = eff_QDriftChann_InitJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90y)
    pulse_mix2 = leftmix@rightmix
    leftmix,rightmix = eff_QDriftChann_InitJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90mx)
    pulse_mix3 = leftmix@rightmix
    leftmix,rightmix = eff_QDriftChann_InitJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90my)
    pulse_mix4 = leftmix@rightmix

    for i in range(Tpts1):
        

        rho_stack1_1.append(pulse_90y@pulse_mix1@rho_stack[i])
        rho_stack1_2.append(pulse_90y@pulse_mix2@rho_stack[i])
        rho_stack1_3.append(pulse_90y@pulse_mix3@rho_stack[i])
        rho_stack1_4.append(pulse_90y@pulse_mix4@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4


def GenFID_TmixFinJumpQFact_cohT1T2(Ham,List_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    
    R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham)*dt1)

    L_dt2 = expm((-1j*Ham)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    #print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    #leftmix, rightmix = eff_QDriftChann_FinJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90x)
    #pulse_mix1 = leftmix@rightmix
    #leftmix, rightmix = eff_QDriftChann_FinJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90y)
    #pulse_mix2 = leftmix@rightmix
    #leftmix,rightmix = eff_QDriftChann_FinJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90mx)
    #pulse_mix3 = leftmix@rightmix
    leftmix,rightmix = eff_QDriftChann_FinJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90y)
    pulse_mix = leftmix@rightmix

    for i in range(Tpts1):
        

        rho_stack1_1.append(pulse_mix@pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_mix@pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_mix@pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_mix@pulse_90my@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4

def GenFID_RemTmixFinJumpQFact_cohT1T2(Ham,List_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    
    R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham)*dt1)

    L_dt2 = expm((-1j*Ham)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    #print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    leftmix, rightmix = eff_QDriftChann_FinJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90y)
    pulse_mix = leftmix@rightmix
    #leftmix, rightmix = eff_QDriftChann_FinJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90y)
    #pulse_mix2 = leftmix
    #leftmix,rightmix = eff_QDriftChann_FinJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90mx)
    #pulse_mix3 = leftmix
    #leftmix,rightmix = eff_QDriftChann_FinJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90my)
    #pulse_mix4 = leftmix

    for i in range(Tpts1):
        

        rho_stack1_1.append(pulse_mix@pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_mix@pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_mix@pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_mix@pulse_90my@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4

def GenFID_RemTmixInitJumpQFact_cohT1T2(Ham,List_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    
    R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham)*dt1)

    L_dt2 = expm((-1j*Ham)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    #print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    leftmix, rightmix = eff_QDriftChann_InitJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90x)
    pulse_mix1 = rightmix
    leftmix, rightmix = eff_QDriftChann_InitJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90y)
    pulse_mix2 = rightmix
    leftmix,rightmix = eff_QDriftChann_InitJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90mx)
    pulse_mix3 = rightmix
    leftmix,rightmix = eff_QDriftChann_InitJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90my)
    pulse_mix4 = rightmix

    for i in range(Tpts1):
        

        rho_stack1_1.append(pulse_90y@pulse_mix1@rho_stack[i])
        rho_stack1_2.append(pulse_90y@pulse_mix2@rho_stack[i])
        rho_stack1_3.append(pulse_90y@pulse_mix3@rho_stack[i])
        rho_stack1_4.append(pulse_90y@pulse_mix4@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4


def GenFID_JumpEOProtocol_cohT1T2(Ham,List_jumps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    
    R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham)*dt1)

    L_dt2 = expm((-1j*Ham)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    #print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))


    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    
    leftmix,rightmix = eff_QDriftEndOfProtocol([Ham]+List_jumps,tmix,Ntmix,pulse_90y)
    pulse_mix = rightmix

    for i in range(Tpts1):
        

        rho_stack1_1.append(pulse_90y@pulse_mix@pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@pulse_mix@pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90y@pulse_mix@pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90y@pulse_mix@pulse_90my@rho_stack[i])


    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    str = np.floor(Tpts1/10)

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        rho1_buff = np.copy(rho1)
        rho2_buff = np.copy(rho2)
        rho3_buff = np.copy(rho3)
        rho4_buff = np.copy(rho4)

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1
            rho1_buff = leftmix@rho1 #probabilistically apply the jump operator at the end of the protocol

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2
            rho2_buff = leftmix@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3
            rho3_buff = leftmix@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
            rho4_buff = leftmix@rho4
        
        if i%str==0:
            print("Processed ", i, "out of", Tpts1, "points")


    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4






In [11]:
text4="""1      (0,0)   (0,0)   (0,0)   (0,0)   
  2      (0,0)   (0,0)   (0,0)   (1,1)   
  3      (0,0)   (0,0)   (0,0)   (1,0)   
  4      (0,0)   (0,0)   (0,0)   (1,-1)  
  5      (0,0)   (0,0)   (1,1)   (0,0)   
  6      (0,0)   (0,0)   (1,1)   (1,1)   
  7      (0,0)   (0,0)   (1,1)   (1,0)   
  8      (0,0)   (0,0)   (1,1)   (1,-1)  
  9      (0,0)   (0,0)   (1,0)   (0,0)   
  10     (0,0)   (0,0)   (1,0)   (1,1)   
  11     (0,0)   (0,0)   (1,0)   (1,0)   
  12     (0,0)   (0,0)   (1,0)   (1,-1)  
  13     (0,0)   (0,0)   (1,-1)  (0,0)   
  14     (0,0)   (0,0)   (1,-1)  (1,1)   
  15     (0,0)   (0,0)   (1,-1)  (1,0)   
  16     (0,0)   (0,0)   (1,-1)  (1,-1)  
  17     (0,0)   (1,1)   (0,0)   (0,0)   
  18     (0,0)   (1,1)   (0,0)   (1,1)   
  19     (0,0)   (1,1)   (0,0)   (1,0)   
  20     (0,0)   (1,1)   (0,0)   (1,-1)  
  21     (0,0)   (1,1)   (1,1)   (0,0)   
  22     (0,0)   (1,1)   (1,1)   (1,1)   
  23     (0,0)   (1,1)   (1,1)   (1,0)   
  24     (0,0)   (1,1)   (1,1)   (1,-1)  
  25     (0,0)   (1,1)   (1,0)   (0,0)   
  26     (0,0)   (1,1)   (1,0)   (1,1)   
  27     (0,0)   (1,1)   (1,0)   (1,0)   
  28     (0,0)   (1,1)   (1,0)   (1,-1)  
  29     (0,0)   (1,1)   (1,-1)  (0,0)   
  30     (0,0)   (1,1)   (1,-1)  (1,1)   
  31     (0,0)   (1,1)   (1,-1)  (1,0)   
  32     (0,0)   (1,1)   (1,-1)  (1,-1)  
  33     (0,0)   (1,0)   (0,0)   (0,0)   
  34     (0,0)   (1,0)   (0,0)   (1,1)   
  35     (0,0)   (1,0)   (0,0)   (1,0)   
  36     (0,0)   (1,0)   (0,0)   (1,-1)  
  37     (0,0)   (1,0)   (1,1)   (0,0)   
  38     (0,0)   (1,0)   (1,1)   (1,1)   
  39     (0,0)   (1,0)   (1,1)   (1,0)   
  40     (0,0)   (1,0)   (1,1)   (1,-1)  
  41     (0,0)   (1,0)   (1,0)   (0,0)   
  42     (0,0)   (1,0)   (1,0)   (1,1)   
  43     (0,0)   (1,0)   (1,0)   (1,0)   
  44     (0,0)   (1,0)   (1,0)   (1,-1)  
  45     (0,0)   (1,0)   (1,-1)  (0,0)   
  46     (0,0)   (1,0)   (1,-1)  (1,1)   
  47     (0,0)   (1,0)   (1,-1)  (1,0)   
  48     (0,0)   (1,0)   (1,-1)  (1,-1)  
  49     (0,0)   (1,-1)  (0,0)   (0,0)   
  50     (0,0)   (1,-1)  (0,0)   (1,1)   
  51     (0,0)   (1,-1)  (0,0)   (1,0)   
  52     (0,0)   (1,-1)  (0,0)   (1,-1)  
  53     (0,0)   (1,-1)  (1,1)   (0,0)   
  54     (0,0)   (1,-1)  (1,1)   (1,1)   
  55     (0,0)   (1,-1)  (1,1)   (1,0)   
  56     (0,0)   (1,-1)  (1,1)   (1,-1)  
  57     (0,0)   (1,-1)  (1,0)   (0,0)   
  58     (0,0)   (1,-1)  (1,0)   (1,1)   
  59     (0,0)   (1,-1)  (1,0)   (1,0)   
  60     (0,0)   (1,-1)  (1,0)   (1,-1)  
  61     (0,0)   (1,-1)  (1,-1)  (0,0)   
  62     (0,0)   (1,-1)  (1,-1)  (1,1)   
  63     (0,0)   (1,-1)  (1,-1)  (1,0)   
  64     (0,0)   (1,-1)  (1,-1)  (1,-1)  
  65     (1,1)   (0,0)   (0,0)   (0,0)   
  66     (1,1)   (0,0)   (0,0)   (1,1)   
  67     (1,1)   (0,0)   (0,0)   (1,0)   
  68     (1,1)   (0,0)   (0,0)   (1,-1)  
  69     (1,1)   (0,0)   (1,1)   (0,0)   
  70     (1,1)   (0,0)   (1,1)   (1,1)   
  71     (1,1)   (0,0)   (1,1)   (1,0)   
  72     (1,1)   (0,0)   (1,1)   (1,-1)  
  73     (1,1)   (0,0)   (1,0)   (0,0)   
  74     (1,1)   (0,0)   (1,0)   (1,1)   
  75     (1,1)   (0,0)   (1,0)   (1,0)   
  76     (1,1)   (0,0)   (1,0)   (1,-1)  
  77     (1,1)   (0,0)   (1,-1)  (0,0)   
  78     (1,1)   (0,0)   (1,-1)  (1,1)   
  79     (1,1)   (0,0)   (1,-1)  (1,0)   
  80     (1,1)   (0,0)   (1,-1)  (1,-1)  
  81     (1,1)   (1,1)   (0,0)   (0,0)   
  82     (1,1)   (1,1)   (0,0)   (1,1)   
  83     (1,1)   (1,1)   (0,0)   (1,0)   
  84     (1,1)   (1,1)   (0,0)   (1,-1)  
  85     (1,1)   (1,1)   (1,1)   (0,0)   
  86     (1,1)   (1,1)   (1,1)   (1,1)   
  87     (1,1)   (1,1)   (1,1)   (1,0)   
  88     (1,1)   (1,1)   (1,1)   (1,-1)  
  89     (1,1)   (1,1)   (1,0)   (0,0)   
  90     (1,1)   (1,1)   (1,0)   (1,1)   
  91     (1,1)   (1,1)   (1,0)   (1,0)   
  92     (1,1)   (1,1)   (1,0)   (1,-1)  
  93     (1,1)   (1,1)   (1,-1)  (0,0)   
  94     (1,1)   (1,1)   (1,-1)  (1,1)   
  95     (1,1)   (1,1)   (1,-1)  (1,0)   
  96     (1,1)   (1,1)   (1,-1)  (1,-1)  
  97     (1,1)   (1,0)   (0,0)   (0,0)   
  98     (1,1)   (1,0)   (0,0)   (1,1)   
  99     (1,1)   (1,0)   (0,0)   (1,0)   
  100    (1,1)   (1,0)   (0,0)   (1,-1)  
  101    (1,1)   (1,0)   (1,1)   (0,0)   
  102    (1,1)   (1,0)   (1,1)   (1,1)   
  103    (1,1)   (1,0)   (1,1)   (1,0)   
  104    (1,1)   (1,0)   (1,1)   (1,-1)  
  105    (1,1)   (1,0)   (1,0)   (0,0)   
  106    (1,1)   (1,0)   (1,0)   (1,1)   
  107    (1,1)   (1,0)   (1,0)   (1,0)   
  108    (1,1)   (1,0)   (1,0)   (1,-1)  
  109    (1,1)   (1,0)   (1,-1)  (0,0)   
  110    (1,1)   (1,0)   (1,-1)  (1,1)   
  111    (1,1)   (1,0)   (1,-1)  (1,0)   
  112    (1,1)   (1,0)   (1,-1)  (1,-1)  
  113    (1,1)   (1,-1)  (0,0)   (0,0)   
  114    (1,1)   (1,-1)  (0,0)   (1,1)   
  115    (1,1)   (1,-1)  (0,0)   (1,0)   
  116    (1,1)   (1,-1)  (0,0)   (1,-1)  
  117    (1,1)   (1,-1)  (1,1)   (0,0)   
  118    (1,1)   (1,-1)  (1,1)   (1,1)   
  119    (1,1)   (1,-1)  (1,1)   (1,0)   
  120    (1,1)   (1,-1)  (1,1)   (1,-1)  
  121    (1,1)   (1,-1)  (1,0)   (0,0)   
  122    (1,1)   (1,-1)  (1,0)   (1,1)   
  123    (1,1)   (1,-1)  (1,0)   (1,0)   
  124    (1,1)   (1,-1)  (1,0)   (1,-1)  
  125    (1,1)   (1,-1)  (1,-1)  (0,0)   
  126    (1,1)   (1,-1)  (1,-1)  (1,1)   
  127    (1,1)   (1,-1)  (1,-1)  (1,0)   
  128    (1,1)   (1,-1)  (1,-1)  (1,-1)  
  129    (1,0)   (0,0)   (0,0)   (0,0)   
  130    (1,0)   (0,0)   (0,0)   (1,1)   
  131    (1,0)   (0,0)   (0,0)   (1,0)   
  132    (1,0)   (0,0)   (0,0)   (1,-1)  
  133    (1,0)   (0,0)   (1,1)   (0,0)   
  134    (1,0)   (0,0)   (1,1)   (1,1)   
  135    (1,0)   (0,0)   (1,1)   (1,0)   
  136    (1,0)   (0,0)   (1,1)   (1,-1)  
  137    (1,0)   (0,0)   (1,0)   (0,0)   
  138    (1,0)   (0,0)   (1,0)   (1,1)   
  139    (1,0)   (0,0)   (1,0)   (1,0)   
  140    (1,0)   (0,0)   (1,0)   (1,-1)  
  141    (1,0)   (0,0)   (1,-1)  (0,0)   
  142    (1,0)   (0,0)   (1,-1)  (1,1)   
  143    (1,0)   (0,0)   (1,-1)  (1,0)   
  144    (1,0)   (0,0)   (1,-1)  (1,-1)  
  145    (1,0)   (1,1)   (0,0)   (0,0)   
  146    (1,0)   (1,1)   (0,0)   (1,1)   
  147    (1,0)   (1,1)   (0,0)   (1,0)   
  148    (1,0)   (1,1)   (0,0)   (1,-1)  
  149    (1,0)   (1,1)   (1,1)   (0,0)   
  150    (1,0)   (1,1)   (1,1)   (1,1)   
  151    (1,0)   (1,1)   (1,1)   (1,0)   
  152    (1,0)   (1,1)   (1,1)   (1,-1)  
  153    (1,0)   (1,1)   (1,0)   (0,0)   
  154    (1,0)   (1,1)   (1,0)   (1,1)   
  155    (1,0)   (1,1)   (1,0)   (1,0)   
  156    (1,0)   (1,1)   (1,0)   (1,-1)  
  157    (1,0)   (1,1)   (1,-1)  (0,0)   
  158    (1,0)   (1,1)   (1,-1)  (1,1)   
  159    (1,0)   (1,1)   (1,-1)  (1,0)   
  160    (1,0)   (1,1)   (1,-1)  (1,-1)  
  161    (1,0)   (1,0)   (0,0)   (0,0)   
  162    (1,0)   (1,0)   (0,0)   (1,1)   
  163    (1,0)   (1,0)   (0,0)   (1,0)   
  164    (1,0)   (1,0)   (0,0)   (1,-1)  
  165    (1,0)   (1,0)   (1,1)   (0,0)   
  166    (1,0)   (1,0)   (1,1)   (1,1)   
  167    (1,0)   (1,0)   (1,1)   (1,0)   
  168    (1,0)   (1,0)   (1,1)   (1,-1)  
  169    (1,0)   (1,0)   (1,0)   (0,0)   
  170    (1,0)   (1,0)   (1,0)   (1,1)   
  171    (1,0)   (1,0)   (1,0)   (1,0)   
  172    (1,0)   (1,0)   (1,0)   (1,-1)  
  173    (1,0)   (1,0)   (1,-1)  (0,0)   
  174    (1,0)   (1,0)   (1,-1)  (1,1)   
  175    (1,0)   (1,0)   (1,-1)  (1,0)   
  176    (1,0)   (1,0)   (1,-1)  (1,-1)  
  177    (1,0)   (1,-1)  (0,0)   (0,0)   
  178    (1,0)   (1,-1)  (0,0)   (1,1)   
  179    (1,0)   (1,-1)  (0,0)   (1,0)   
  180    (1,0)   (1,-1)  (0,0)   (1,-1)  
  181    (1,0)   (1,-1)  (1,1)   (0,0)   
  182    (1,0)   (1,-1)  (1,1)   (1,1)   
  183    (1,0)   (1,-1)  (1,1)   (1,0)   
  184    (1,0)   (1,-1)  (1,1)   (1,-1)  
  185    (1,0)   (1,-1)  (1,0)   (0,0)   
  186    (1,0)   (1,-1)  (1,0)   (1,1)   
  187    (1,0)   (1,-1)  (1,0)   (1,0)   
  188    (1,0)   (1,-1)  (1,0)   (1,-1)  
  189    (1,0)   (1,-1)  (1,-1)  (0,0)   
  190    (1,0)   (1,-1)  (1,-1)  (1,1)   
  191    (1,0)   (1,-1)  (1,-1)  (1,0)   
  192    (1,0)   (1,-1)  (1,-1)  (1,-1)  
  193    (1,-1)  (0,0)   (0,0)   (0,0)   
  194    (1,-1)  (0,0)   (0,0)   (1,1)   
  195    (1,-1)  (0,0)   (0,0)   (1,0)   
  196    (1,-1)  (0,0)   (0,0)   (1,-1)  
  197    (1,-1)  (0,0)   (1,1)   (0,0)   
  198    (1,-1)  (0,0)   (1,1)   (1,1)   
  199    (1,-1)  (0,0)   (1,1)   (1,0)   
  200    (1,-1)  (0,0)   (1,1)   (1,-1)  
  201    (1,-1)  (0,0)   (1,0)   (0,0)   
  202    (1,-1)  (0,0)   (1,0)   (1,1)   
  203    (1,-1)  (0,0)   (1,0)   (1,0)   
  204    (1,-1)  (0,0)   (1,0)   (1,-1)  
  205    (1,-1)  (0,0)   (1,-1)  (0,0)   
  206    (1,-1)  (0,0)   (1,-1)  (1,1)   
  207    (1,-1)  (0,0)   (1,-1)  (1,0)   
  208    (1,-1)  (0,0)   (1,-1)  (1,-1)  
  209    (1,-1)  (1,1)   (0,0)   (0,0)   
  210    (1,-1)  (1,1)   (0,0)   (1,1)   
  211    (1,-1)  (1,1)   (0,0)   (1,0)   
  212    (1,-1)  (1,1)   (0,0)   (1,-1)  
  213    (1,-1)  (1,1)   (1,1)   (0,0)   
  214    (1,-1)  (1,1)   (1,1)   (1,1)   
  215    (1,-1)  (1,1)   (1,1)   (1,0)   
  216    (1,-1)  (1,1)   (1,1)   (1,-1)  
  217    (1,-1)  (1,1)   (1,0)   (0,0)   
  218    (1,-1)  (1,1)   (1,0)   (1,1)   
  219    (1,-1)  (1,1)   (1,0)   (1,0)   
  220    (1,-1)  (1,1)   (1,0)   (1,-1)  
  221    (1,-1)  (1,1)   (1,-1)  (0,0)   
  222    (1,-1)  (1,1)   (1,-1)  (1,1)   
  223    (1,-1)  (1,1)   (1,-1)  (1,0)   
  224    (1,-1)  (1,1)   (1,-1)  (1,-1)  
  225    (1,-1)  (1,0)   (0,0)   (0,0)   
  226    (1,-1)  (1,0)   (0,0)   (1,1)   
  227    (1,-1)  (1,0)   (0,0)   (1,0)   
  228    (1,-1)  (1,0)   (0,0)   (1,-1)  
  229    (1,-1)  (1,0)   (1,1)   (0,0)   
  230    (1,-1)  (1,0)   (1,1)   (1,1)   
  231    (1,-1)  (1,0)   (1,1)   (1,0)   
  232    (1,-1)  (1,0)   (1,1)   (1,-1)  
  233    (1,-1)  (1,0)   (1,0)   (0,0)   
  234    (1,-1)  (1,0)   (1,0)   (1,1)   
  235    (1,-1)  (1,0)   (1,0)   (1,0)   
  236    (1,-1)  (1,0)   (1,0)   (1,-1)  
  237    (1,-1)  (1,0)   (1,-1)  (0,0)   
  238    (1,-1)  (1,0)   (1,-1)  (1,1)   
  239    (1,-1)  (1,0)   (1,-1)  (1,0)   
  240    (1,-1)  (1,0)   (1,-1)  (1,-1)  
  241    (1,-1)  (1,-1)  (0,0)   (0,0)   
  242    (1,-1)  (1,-1)  (0,0)   (1,1)   
  243    (1,-1)  (1,-1)  (0,0)   (1,0)   
  244    (1,-1)  (1,-1)  (0,0)   (1,-1)  
  245    (1,-1)  (1,-1)  (1,1)   (0,0)   
  246    (1,-1)  (1,-1)  (1,1)   (1,1)   
  247    (1,-1)  (1,-1)  (1,1)   (1,0)   
  248    (1,-1)  (1,-1)  (1,1)   (1,-1)  
  249    (1,-1)  (1,-1)  (1,0)   (0,0)   
  250    (1,-1)  (1,-1)  (1,0)   (1,1)   
  251    (1,-1)  (1,-1)  (1,0)   (1,0)   
  252    (1,-1)  (1,-1)  (1,0)   (1,-1)  
  253    (1,-1)  (1,-1)  (1,-1)  (0,0)   
  254    (1,-1)  (1,-1)  (1,-1)  (1,1)   
  255    (1,-1)  (1,-1)  (1,-1)  (1,0)   
  256    (1,-1)  (1,-1)  (1,-1)  (1,-1)"""



In [83]:
data_ala = read_spinach_info(text4)

basis_ala = build_list_ISTs(data_ala)
prefacts,Symb_ALA_basis = build_symbolic_list_ISTs(data_ala)

#Normbasis = NormalizeBasis(basis,n_qubits=4,checkOrth=True) I have verified the orthonormalization of the basis
Normbasis_ala = NormalizeBasis(basis_ala,n_qubits=4,checkOrth=False)
Normbasis_ala = np.array(Normbasis_ala)

f = open('./data/ALA_JOps.pk','rb')
dat = pickle.load(f)

JumpOps = dat['JOps']

Rtrunc = sum(JumpOps)


In [81]:
loadMat = spio.loadmat('./data/NOESYdata_ALA_withGradients.mat',squeeze_me=True)

t_grid1 = loadMat['p']['time_grid1'].item()
t_grid2 = loadMat['p']['time_grid2'].item()

R_ala = loadMat['p']['R'].item()
H_ala = loadMat['p']['H'].item().toarray()

rho0 = np.array(loadMat['p']['rho0'].item().toarray())

coil = np.array(loadMat['p']['coil'].item())

Lx = loadMat['p']['Lx'].item().toarray() 
Ly = loadMat['p']['Ly'].item().toarray() 

###Dynamical evolution for calculation of 2D spectra...
#Tpts1 = len(t_grid1)
#Tpts2 = len(t_grid2)

#Parameters taken from Spinach script
tmix = 1.0
dt1 = 5e-4
dt2 = 5e-4
#dt1 = 0.00512
#dt2 = 0.00512
Tpts1 = 512
Tpts2 = 512

T1 = Tpts1*dt1
T2 = Tpts2*dt2

Ntmix = 2631
####Generation of the different FIDs that consider the commutation through of the single jump operator either to the T1 or T2 timescales of the protocol; and the effective elimination of the 
# mixing time 

#fid1_JtoT1, fid2_JtoT1, fid3_JtoT1, fid4_JtoT1 = GenFID_TmixInitJumpQFact_cohT1T2(H_ala,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly)

#fid1_JtoT2, fid2_JtoT2, fid3_JtoT2, fid4_JtoT2 = GenFID_TmixFinJumpQFact_cohT1T2(H_ala,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly)

#fid1_JtoT1Rem, fid2_JtoT1Rem, fid3_JtoT1Rem, fid4_JtoT1Rem = GenFID_RemTmixInitJumpQFact_cohT1T2(H_ala,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly)

#fid1_JtoT2Rem, fid2_JtoT2Rem, fid3_JtoT2Rem, fid4_JtoT2Rem = GenFID_RemTmixFinJumpQFact_cohT1T2(H_ala,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly)

#fid1_noTmix, fid2_noTmix, fid3_noTmix, fid4_noTmix = GenFID_NoTmix_cohT1T2(H_ala,JumpOps,T1,T2,rho0,coil,dt1,dt2,Lx,Ly)

fid1_coh, fid2_coh, fid3_coh, fid4_coh = GenFID_cohevol(H_ala,T1,T2,rho0,coil,tmix,dt1,dt2,Lx,Ly)







/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_25765/3325980156.py:180: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_1[j,i] = np.dot(coil,rho1)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_25765/3325980156.py:183: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_2[j,i] = np.dot(coil,rho2)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_25765/3325980156.py:186: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_tem

In [62]:

pulse_90x = expm(-1j*Lx*np.pi/2)
pulse_90mx = expm(1j*Lx*np.pi/2)
pulse_90y = expm(-1j*Ly*np.pi/2)

rho_t1 = expm(-1j*H_ala*0.256)@pulse_90x@rho0.flatten()
#rho_t1 = pulse_90x@rho0.flatten()

np.vdot(rho0,pulse_90mx@rho_t1)


(0.549086109702331-5.96404801196445e-16j)

In [82]:

cos_fid = fid1_coh - fid3_coh
sin_fid = fid2_coh - fid4_coh

savemat('ALA_truncR_FIDcos_coh.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_coh.mat', {'FID_sin': sin_fid})

In [20]:
cos_fid = fid1_noTmix - fid3_noTmix
sin_fid = fid2_noTmix - fid4_noTmix

savemat('ALA_truncR_FIDcos_noTmix.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_noTmix.mat', {'FID_sin': sin_fid})

In [48]:
###Saving for postprocessing in Matlab
cos_fid = fid1_JtoT1 - fid3_JtoT1
sin_fid = fid2_JtoT1 - fid4_JtoT1

savemat('ALA_truncR_FIDcos_JtoT1.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_JtoT1.mat', {'FID_sin': sin_fid})


cos_fid = fid1_JtoT2 - fid3_JtoT2
sin_fid = fid2_JtoT2 - fid4_JtoT2

savemat('ALA_truncR_FIDcos_JtoT2.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_JtoT2.mat', {'FID_sin': sin_fid})

cos_fid = fid1_JtoT1Rem - fid3_JtoT1Rem
sin_fid = fid2_JtoT1Rem - fid4_JtoT1Rem

savemat('ALA_truncR_FIDcos_JtoT1Rem.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_JtoT1Rem.mat', {'FID_sin': sin_fid})

cos_fid = fid1_JtoT2Rem - fid3_JtoT2Rem
sin_fid = fid2_JtoT2Rem - fid4_JtoT2Rem

savemat('ALA_truncR_FIDcos_JtoT2Rem.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_JtoT2Rem.mat', {'FID_sin': sin_fid})


In [44]:
####The "commutation through" of the jump operator to T1 does not affect the spectrum (in principle, it must be exactly the same, but I noticed a small change in a few signals)

#Subsequent elimination of the mixing time leads to qualitative changes, which is consistent with eliminating the mixing time altogether of the protocol

#TODO: obtain the FID signals that come out from the probabilistic application of the jump operator at the end of the signal...
fid1_J_EOP,fid2_J_EOP,fid3_J_EOP,fid4_J_EOP =GenFID_JumpEOProtocol_cohT1T2(H_ala,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly)



Number of Trotter steps for deterministic simulation of T1 512
Number of Trotter steps for deterministic simulation of T2 512
Number of Trotter steps for QDrift simulation of tmix 2631


/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_25765/2125240492.py:716: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_1[j,i] = np.dot(coil,rho1)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_25765/2125240492.py:720: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_2[j,i] = np.dot(coil,rho2)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_25765/2125240492.py:724: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_tem

Processed  0 out of 512 points
Processed  51 out of 512 points
Processed  102 out of 512 points
Processed  153 out of 512 points
Processed  204 out of 512 points
Processed  255 out of 512 points
Processed  306 out of 512 points
Processed  357 out of 512 points
Processed  408 out of 512 points
Processed  459 out of 512 points
Processed  510 out of 512 points


In [45]:
cos_fid = fid1_J_EOP - fid3_J_EOP
sin_fid = fid2_J_EOP - fid4_J_EOP

savemat('ALA_truncR_FIDcos_J_EOP.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_J_EOP.mat', {'FID_sin': sin_fid})


# Turning off J couplings of the Hamiltonian

In [84]:
loadMat = spio.loadmat('../matlab_analysis/NOESYdata_ALA_NoJs.mat',squeeze_me=True)


H_ala_noJs = loadMat['p']['H'].item().toarray()



In [78]:
fid1_noJs, fid2_noJs, fid3_noJs, fid4_noJs =  GenFID_cohevol_noJsInTmix(H_ala,H_ala_noJs,T1,T2,rho0,coil,tmix,dt1,dt2,Lx,Ly)

/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_25765/3325980156.py:265: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_1[j,i] = np.dot(coil,rho1)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_25765/3325980156.py:268: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_2[j,i] = np.dot(coil,rho2)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_25765/3325980156.py:271: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_tem

In [79]:
cos_fid = fid1_noJs - fid3_noJs
sin_fid = fid2_noJs - fid4_noJs

savemat('ALA_truncR_FIDcos_TmixNoJs.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_TmixNoJs.mat', {'FID_sin': sin_fid})

In [85]:
np.linalg.norm(H_ala-H_ala_noJs)

1700.6811411997246

In [86]:
fid1_allNoJs,fid2_allNoJs, fid3_allNoJs, fid4_allNoJs = GenFID_cohevol_noJsInTmix(H_ala_noJs,H_ala_noJs,T1,T2,rho0,coil,tmix,dt1,dt2,Lx,Ly)

/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_25765/3325980156.py:265: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_1[j,i] = np.dot(coil,rho1)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_25765/3325980156.py:268: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_temp_2[j,i] = np.dot(coil,rho2)
/var/folders/rm/xdxcjr7n7_13f9m1905zyf6c0000gq/T/ipykernel_25765/3325980156.py:271: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fid_tem

In [87]:
cos_fid = fid1_allNoJs - fid3_allNoJs
sin_fid = fid2_allNoJs - fid4_allNoJs

savemat('ALA_truncR_FIDcos_AllNoJs.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_AllNoJs.mat', {'FID_sin': sin_fid})

# Inclusion of field gradient in the calculations and using updated Parameters

In [197]:
# For a low frequency resolution experiment, where the effect of J-couplings cannot be resolved, the conjecture is that we can prescind from the mixing time phase 
# during the NOESY protocol. To prove this, we can simply implement a gradient field protocol in the spectrum and compare the results of the simulation when including or
#eliminating the mixing time, (do we also need to include the J couplings during t1 and t2? It seems this is the case, as omitting J couplings during these timescales eliminates cross-peaks)
#From this we infer that the scalar couplings induce magnetization transfer between spins during t1 and t2 timescales

"""
def FieldGrad(Npts_grad,rho_stack,dim,Lz):
    
    Inclusion of magnetic field gradient in the simulation. Warning: it modifies and returns the rho_stack array
    

    phi = np.linspace(0,2*np.pi,Npts_grad)

    gradmat = np.zeros(dim,dim,Npts_grad)

    for k in range(Npts_grad):
        gradmat[:,:,k] = expm(-1j*Lz*phi[k])


    for j in range(rho_stack.shape[1]): #iterating over the number of time points....
        rho_temp = rho_stack[:,j,0]
        rho_sum = 0
        for i in range(Npts_grad):
            rho_sum = rho_sum + gradmat[:,:,i]@rho_temp

        rho_sum = rho_sum/Npts_grad
        rho_stack[:,j,0] = rho_sum

        rho_temp = rho_stack[:,j,1]
        rho_sum = 0
        for i in range(Npts_grad):
            rho_sum = rho_sum + gradmat[:,:,i]@rho_temp

        rho_sum = rho_sum/Npts_grad
        rho_stack[:,j,1] = rho_sum

        rho_temp = rho_stack[:,j,2]
        rho_sum = 0
        for i in range(Npts_grad):
            rho_sum = rho_sum + gradmat[:,:,i]@rho_temp

        rho_sum = rho_sum/Npts_grad
        rho_stack[:,j,2] = rho_sum

        rho_temp = rho_stack[:,j,3]
        rho_sum = 0
        for i in range(Npts_grad):
            rho_sum = rho_sum + gradmat[:,:,i]@rho_temp

        rho_sum = rho_sum/Npts_grad
        rho_stack[:,j,3] = rho_sum

    return rho_stack
"""

def FieldGrad(Npts_grad,rho_stack,dim,Lz):
    """ 
    Inclusion of magnetic field gradient in the simulation. Warning: it modifies and returns the rho_stack array
    """

    phi = np.linspace(0,2*np.pi,Npts_grad)

    gradmat = np.zeros([dim,dim,Npts_grad],dtype=complex)

    for k in range(Npts_grad):
        gradmat[:,:,k] = expm(-1j*Lz*phi[k])


    for j in range(len(rho_stack)): #iterating over the number of time points....
        rho_temp = rho_stack[j]
        rho_sum = 0
        for i in range(Npts_grad):
            rho_sum = rho_sum + gradmat[:,:,i]@rho_temp

        rho_sum = rho_sum/Npts_grad
        rho_stack[j] = rho_sum


    return rho_stack

###TODO: test this function....
def GenFID_cohevol_GradField(Ham,T1,T2,rho0,coil,tmix,dt1,dt2,Lx,Ly,Lz):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    rho0 = rho0.flatten()

    Npts_grad = 100
    dim = rho0.shape[0]
    
    #R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham)*dt1)

    L_dt2 = expm((-1j*Ham)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of points for simulation of T1",Tpts1)
    print("Number of points for simulation of T2",Tpts2)
    #print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    #print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))

    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)

    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    #TODO: change this accordingly to include dissipation....
    #leftmix,rightmix = eff_QDriftChann_FinJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90y)
    #pulse_mix = leftmix@rightmix
    pulse_mix = expm(-1j*Ham*tmix)


    ###Aplication of second 90 deg pulse

    print("Application of second 90 deg pulse")
    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90my@rho_stack[i])


    #####Aplication of gradient...
    rho_stack1_1 = FieldGrad(Npts_grad,rho_stack1_1,dim,Lz)
    rho_stack1_2 = FieldGrad(Npts_grad,rho_stack1_2,dim,Lz)
    rho_stack1_3 = FieldGrad(Npts_grad,rho_stack1_3,dim,Lz)
    rho_stack1_4 = FieldGrad(Npts_grad,rho_stack1_4,dim,Lz)

    print("Mixing time")
    ###aplication of mixing
    for i in range(Tpts1):
        rho_stack1_1[i] = pulse_mix@rho_stack1_1[i]
        rho_stack1_2[i] = pulse_mix@rho_stack1_2[i]
        rho_stack1_3[i] = pulse_mix@rho_stack1_3[i]
        rho_stack1_4[i] = pulse_mix@rho_stack1_4[i]

    #aplication of gradient...
    rho_stack1_1 = FieldGrad(Npts_grad,rho_stack1_1,dim,Lz)
    rho_stack1_2 = FieldGrad(Npts_grad,rho_stack1_2,dim,Lz)
    rho_stack1_3 = FieldGrad(Npts_grad,rho_stack1_3,dim,Lz)
    rho_stack1_4 = FieldGrad(Npts_grad,rho_stack1_4,dim,Lz)

    print("Application of third 90 deg pulse")
    #application of third pulse...
    for i in range(Tpts1):
        rho_stack1_1[i] = pulse_90y@rho_stack1_1[i]
        rho_stack1_2[i] = pulse_90y@rho_stack1_2[i]
        rho_stack1_3[i] = pulse_90y@rho_stack1_3[i]
        rho_stack1_4[i] = pulse_90y@rho_stack1_4[i]



    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4
    
def GenFID_cohevol_noJsinTmix_GradField(Ham,Ham_noJs,T1,T2,rho0,coil,tmix,dt1,dt2,Lx,Ly,Lz):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    rho0 = rho0.flatten()

    Npts_grad = 100
    dim = rho0.shape[0]
    
    #R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham)*dt1)

    L_dt2 = expm((-1j*Ham)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    #print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    #print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))

    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)

    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    #TODO: change this accordingly to include dissipation....
    #leftmix,rightmix = eff_QDriftChann_FinJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90y)
    #pulse_mix = leftmix@rightmix
    pulse_mix = expm(-1j*Ham_noJs*tmix)


    ###Aplication of second 90 deg pulse

    print("Application of second 90 deg pulse")
    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90my@rho_stack[i])


    #####Aplication of gradient...
    rho_stack1_1 = FieldGrad(Npts_grad,rho_stack1_1,dim,Lz)
    rho_stack1_2 = FieldGrad(Npts_grad,rho_stack1_2,dim,Lz)
    rho_stack1_3 = FieldGrad(Npts_grad,rho_stack1_3,dim,Lz)
    rho_stack1_4 = FieldGrad(Npts_grad,rho_stack1_4,dim,Lz)

    print("Mixing time")
    ###aplication of mixing
    for i in range(Tpts1):
        rho_stack1_1.append(pulse_mix@rho_stack[i])
        rho_stack1_2.append(pulse_mix@rho_stack[i])
        rho_stack1_3.append(pulse_mix@rho_stack[i])
        rho_stack1_4.append(pulse_mix@rho_stack[i])

    #aplication of gradient...
    rho_stack1_1 = FieldGrad(Npts_grad,rho_stack1_1,dim,Lz)
    rho_stack1_2 = FieldGrad(Npts_grad,rho_stack1_2,dim,Lz)
    rho_stack1_3 = FieldGrad(Npts_grad,rho_stack1_3,dim,Lz)
    rho_stack1_4 = FieldGrad(Npts_grad,rho_stack1_4,dim,Lz)

    print("Application of third 90 deg pulse")
    #application of third pulse...
    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90y@rho_stack[i])
        rho_stack1_2.append(pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90y@rho_stack[i])
        rho_stack1_4.append(pulse_90y@rho_stack[i])



    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4



def GenFID_cohevol_noTmix_GradField(Ham,Ham_noJs,T1,T2,rho0,coil,tmix,dt1,dt2,Lx,Ly,Lz):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    rho0 = rho0.flatten()

    Npts_grad = 100
    dim = rho0.shape[0]
    
    #R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    L_dt1 = expm((-1j*Ham)*dt1)

    L_dt2 = expm((-1j*Ham)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of Trotter steps for deterministic simulation of T1",Tpts1)
    print("Number of Trotter steps for deterministic simulation of T2",Tpts2)
    #print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    #print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))

    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)

    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    #TODO: change this accordingly to include dissipation....
    #leftmix,rightmix = eff_QDriftChann_FinJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90y)
    #pulse_mix = leftmix@rightmix
    pulse_mix = expm(-1j*Ham_noJs*tmix)


    ###Aplication of second 90 deg pulse

    print("Application of second 90 deg pulse")
    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90my@rho_stack[i])


    #####Aplication of gradient...
    rho_stack1_1 = FieldGrad(Npts_grad,rho_stack1_1,dim,Lz)
    rho_stack1_2 = FieldGrad(Npts_grad,rho_stack1_2,dim,Lz)
    rho_stack1_3 = FieldGrad(Npts_grad,rho_stack1_3,dim,Lz)
    rho_stack1_4 = FieldGrad(Npts_grad,rho_stack1_4,dim,Lz)

    #print("Mixing time")
    ###aplication of mixing
    #for i in range(Tpts1):
    #    rho_stack1_1.append(pulse_mix@rho_stack[i])
    #    rho_stack1_2.append(pulse_mix@rho_stack[i])
    #    rho_stack1_3.append(pulse_mix@rho_stack[i])
    #    rho_stack1_4.append(pulse_mix@rho_stack[i])

    #aplication of gradient...
    #rho_stack1_1 = FieldGrad(Npts_grad,rho_stack1_1,dim,Lz)
    #rho_stack1_2 = FieldGrad(Npts_grad,rho_stack1_2,dim,Lz)
    #rho_stack1_3 = FieldGrad(Npts_grad,rho_stack1_3,dim,Lz)
    #rho_stack1_4 = FieldGrad(Npts_grad,rho_stack1_4,dim,Lz)

    print("Application of third 90 deg pulse")
    #application of third pulse...
    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90y@rho_stack[i])
        rho_stack1_2.append(pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90y@rho_stack[i])
        rho_stack1_4.append(pulse_90y@rho_stack[i])



    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4

#####Function to compute FID by neglecting mixing time and using a probabilistic application of the jump operator at the beginning of t2...
def GenFID_SingJump_noTmix_GradField(Ham,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly,Lz):
    """
    Implement a first-order deterministic Trotter simulation of the FID NOESY signals, to numerically explore optimization of number of Trotter steps needed in the simulation
    for a target precision
    Args:
    Ham: matrix representation of the "coherent" part of the Liouvillian superoperator
    List_jumps: list of the matrix representation of the jump operators
    T1: maximal simulation time for the first phase of the protocol
    T2: maximal simulation time for the second phase of the protocol
    rho0: initial density matrix vector
    coil: vector that represents the obervable we trace over to compute FID
    Ntmix: number of samples to drawn in QDrift for the simulation of the system for tmix
    """
    rho0 = rho0.flatten()

    Npts_grad = 100
    dim = rho0.shape[0]
    
    #R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    R = sum(JumpOps)

    L_dt1 = expm((-1j*Ham+R)*dt1)

    L_dt2 = expm((-1j*Ham+R)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of points for T1",Tpts1)
    print("Number of points for T2",Tpts2)
    #print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    #print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))

    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        rho_temp = np.dot(L_dt1,rho_temp)
        rho_stack.append(rho_temp)

    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    #TODO: change this accordingly to include dissipation....
    #leftmix,rightmix = eff_QDriftChann_FinJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90y)
    #pulse_mix = leftmix@rightmix

    #This is the last pulse that mimics the relaxation ocurring during the mixing time, thus eliminating the need of including mixing time
    eff_pulse_90y = get_last_QDriftJump([Ham]+JumpOps,tmix,Ntmix,pulse_90y)
    

    ###Aplication of second 90 deg pulse

    print("Application of second 90 deg pulse")
    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90my@rho_stack[i])


    #####Aplication of gradient...
    rho_stack1_1 = FieldGrad(Npts_grad,rho_stack1_1,dim,Lz)
    rho_stack1_2 = FieldGrad(Npts_grad,rho_stack1_2,dim,Lz)
    rho_stack1_3 = FieldGrad(Npts_grad,rho_stack1_3,dim,Lz)
    rho_stack1_4 = FieldGrad(Npts_grad,rho_stack1_4,dim,Lz)

    #print("Mixing time")
    ###aplication of mixing
    #for i in range(Tpts1):
    #    rho_stack1_1.append(pulse_mix@rho_stack[i])
    #    rho_stack1_2.append(pulse_mix@rho_stack[i])
    #    rho_stack1_3.append(pulse_mix@rho_stack[i])
    #    rho_stack1_4.append(pulse_mix@rho_stack[i])

    #aplication of gradient...
    #rho_stack1_1 = FieldGrad(Npts_grad,rho_stack1_1,dim,Lz)
    #rho_stack1_2 = FieldGrad(Npts_grad,rho_stack1_2,dim,Lz)
    #rho_stack1_3 = FieldGrad(Npts_grad,rho_stack1_3,dim,Lz)
    #rho_stack1_4 = FieldGrad(Npts_grad,rho_stack1_4,dim,Lz)

    print("Application of third 90 deg pulse")
    #application of third pulse...
    for i in range(Tpts1):
        rho_stack1_1[i] = eff_pulse_90y@rho_stack1_1[i]
        rho_stack1_2[i] = eff_pulse_90y@rho_stack1_2[i]
        rho_stack1_3[i] = eff_pulse_90y@rho_stack1_3[i]
        rho_stack1_4[i] = eff_pulse_90y@rho_stack1_4[i]



    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    

    for i in range(Tpts1):
        rho1 = rho_stack1_1[i]
        rho2 = rho_stack1_2[i]
        rho3 = rho_stack1_3[i]
        rho4 = rho_stack1_4[i]

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1)
            rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2)
            rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3)
            rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4)
            rho4 = L_dt2@rho4
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4


def GenFID_SingJump_noTmix_GradField_SetMaxDisc(Ham,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly,Lz,Nmax):
    """
    We aim to perform the simulation by setting a maximum number of samples from a QDrift channel to perform the simulation of t1 and t2 timescales
    to perform the simulation of NOESY spectrum. 
    """
    rho0 = rho0.flatten()

    Npts_grad = 100
    dim = rho0.shape[0]
    
    #R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    #R = sum(JumpOps)

    #L_dt1 = expm((-1j*Ham+R)*dt1)

    #L_dt2 = expm((-1j*Ham+R)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of points for T1",Tpts1)
    print("Number of points for T2",Tpts2)
    #print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    #print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))

    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    ###Getting the list of operators and probabilities for the QDrift channel...
    Norm_ops, List_weights = Normalize_and_weightOps([Ham]+JumpOps)

    List_weights = np.array(List_weights)
    Gamma =  np.sum(List_weights)

    pks = (1.0/Gamma)*List_weights

    #QDriftEvol_fromNormOps(Norm_ops,pks,Gamma,time_evol,Nsteps,rho)

    #rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        ####after the number of time points exceeds the maximum, we keep the number of samples in the QDrift channel fixed...
        if i > Nmax:
            #rho_t = QDriftEvol([Ham]+JumpOps,i*dt1,Nmax,rho0)
            rho_t = QDriftEvol_fromNormOps(Norm_ops,pks,Gamma,i*dt1,Nmax,rho0)

            #rho_temp = np.dot(L_dt1,rho_temp)
            rho_stack.append(rho_t)

        else:
            #rho_t = QDriftEvol([Ham]+JumpOps,i*dt1,i,rho0)
            rho_t = QDriftEvol_fromNormOps(Norm_ops,pks,Gamma,i*dt1,i,rho0)

            #rho_temp = np.dot(L_dt1,rho_temp)
            rho_stack.append(rho_t)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    #TODO: change this accordingly to include dissipation....
    #leftmix,rightmix = eff_QDriftChann_FinJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90y)
    #pulse_mix = leftmix@rightmix

    #This is the last pulse that mimics the relaxation ocurring during the mixing time, thus eliminating the need of including mixing time
    eff_pulse_90y = get_last_QDriftJump([Ham]+JumpOps,tmix,Ntmix,pulse_90y)
    

    ###Aplication of second 90 deg pulse

    print("Application of second 90 deg pulse")
    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90my@rho_stack[i])


    #####Aplication of gradient...
    rho_stack1_1 = FieldGrad(Npts_grad,rho_stack1_1,dim,Lz)
    rho_stack1_2 = FieldGrad(Npts_grad,rho_stack1_2,dim,Lz)
    rho_stack1_3 = FieldGrad(Npts_grad,rho_stack1_3,dim,Lz)
    rho_stack1_4 = FieldGrad(Npts_grad,rho_stack1_4,dim,Lz)

    #print("Mixing time")
    ###aplication of mixing
    #for i in range(Tpts1):
    #    rho_stack1_1.append(pulse_mix@rho_stack[i])
    #    rho_stack1_2.append(pulse_mix@rho_stack[i])
    #    rho_stack1_3.append(pulse_mix@rho_stack[i])
    #    rho_stack1_4.append(pulse_mix@rho_stack[i])

    #aplication of gradient...
    #rho_stack1_1 = FieldGrad(Npts_grad,rho_stack1_1,dim,Lz)
    #rho_stack1_2 = FieldGrad(Npts_grad,rho_stack1_2,dim,Lz)
    #rho_stack1_3 = FieldGrad(Npts_grad,rho_stack1_3,dim,Lz)
    #rho_stack1_4 = FieldGrad(Npts_grad,rho_stack1_4,dim,Lz)

    print("Application of third 90 deg pulse")
    #application of third pulse...
    for i in range(Tpts1):
        rho_stack1_1[i] = eff_pulse_90y@rho_stack1_1[i]
        rho_stack1_2[i] = eff_pulse_90y@rho_stack1_2[i]
        rho_stack1_3[i] = eff_pulse_90y@rho_stack1_3[i]
        rho_stack1_4[i] = eff_pulse_90y@rho_stack1_4[i]



    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    
    for i in range(Tpts1):
        rho1 = np.copy(rho_stack1_1[i])
        rho2 = np.copy(rho_stack1_2[i])
        rho3 = np.copy(rho_stack1_3[i])
        rho4 = np.copy(rho_stack1_4[i])

        rho1_t2 = np.copy(rho1)
        rho2_t2 = np.copy(rho2)
        rho3_t2 = np.copy(rho3)
        rho4_t2 = np.copy(rho4)

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1_t2)

            if j > Nmax:
                #rho1_t2 = QDriftEvol([Ham]+JumpOps,j*dt2,Nmax,rho1)
                rho1_t2 = QDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,Nmax,rho1)
            else:
                rho1_t2 = QDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,j,rho1)
            
            #rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2_t2)
            if j > Nmax:
                rho2_t2 = QDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,Nmax,rho2)
            else:
                rho2_t2 = QDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,j,rho2)
            #rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3_t2)
            if j > Nmax:
                rho3_t2 = QDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,Nmax,rho3)
            else:
                rho3_t2 = QDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,j,rho3)

            #rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4_t2)
            if j > Nmax:
                rho4_t2 = QDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,Nmax,rho4)
            else:
                rho4_t2 = QDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,j,rho4)

            #rho4 = L_dt2@rho4
        print("Finishing iteration: ",i)
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4



In [150]:
####TODO: loading the coherent part of the Liouvillian, that can be generated with and without J couplings from MATLAB
#2) generate the list of jump operators with the new parameters of the Hamiltonian, only needed if the distances between spins is different than the ones used for Alanine so far
#3) if there is a match with the reference Hamiltonian, then we can implement the version without mixing time...


#TODO: the jump operator rates are expected to be slightly different from the ones we have previously computed, we can correct for the this small difference later. The only difference is to use the 
# updated correlation time tc 18.8e-12


loadMat = spio.loadmat('./data/NOESYdata_ALA_experiment.mat',squeeze_me=True)

#R_ala = loadMat['p']['R'].item()
H_ala_exp = loadMat['p']['H'].item().toarray()
Lx = loadMat['p']['Lx'].item().toarray() 
Ly = loadMat['p']['Ly'].item().toarray() 
Lz = loadMat['p']['Lz'].item().toarray()
rho0= loadMat['p']['rho0'].item()
coil  = loadMat['p']['coil'].item().toarray().flatten()
Tpts = 1024#loadMat['p']['Tpts'].item()
#dt = loadMat['p']['dt'].item()
dt1 = 0.3126e-3
dt2 = 0.2080e-3

T1 = dt1*Tpts
T2 = dt2*Tpts
tmix = 0.80

#Load the list of ZZ jump operators....
f= open('./data/ALA_expGradField_JOps.pk')
dat = pickle.load(f)

JumpOps = dat['JumpOps']


#fid1_cohGrad, fid2_cohGrad, fid3_cohGrad, fid4_cohGrad = GenFID_cohevol_GradField(H_ala_exp,T1,T2,rho0,coil,tmix,dt1,dt2,Lx,Ly,Lz)




Number of points for simulation of T1 1024
Number of points for simulation of T2 1024
Application of second 90 deg pulse
Mixing time
Application of third 90 deg pulse


In [179]:
###saving data...
cos_fid = fid1_cohGrad - fid3_cohGrad
sin_fid = fid2_cohGrad - fid4_cohGrad

savemat('ALA_truncR_FIDcos_cohGrad.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_cohGrad.mat', {'FID_sin': sin_fid})


In [191]:
fid1_effJ, fid2_effJ, fid3_effJ, fid4_effJ = GenFID_SingJump_noTmix_GradField(H_ala_exp,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly,Lz)

Number of points for T1 1024
Number of points for T2 1024
Application of second 90 deg pulse
Application of third 90 deg pulse


In [193]:
cos_fid = fid1_effJ - fid3_effJ
sin_fid = fid2_effJ - fid4_effJ

savemat('ALA_truncR_FIDcos_effJGrad.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_effJGrad.mat', {'FID_sin': sin_fid})

In [198]:
Nmax = 10
fid1_effJN10, fid2_effJN10, fid3_effJN10, fid4_effJN10 = GenFID_SingJump_noTmix_GradField_SetMaxDisc(H_ala_exp,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly,Lz,Nmax)

Number of points for T1 1024
Number of points for T2 1024
Application of second 90 deg pulse
Application of third 90 deg pulse
Finishing iteration:  0


KeyboardInterrupt: 

# Number of layer estimation for the computation of NOESY spectrum

In [ ]:
#perform optimistic and pessimistic estimation of number of layers for the implementation of a NOESY protocol in a quantum device...
# under the following simplifications. For the most optimistic case:
#1) we can apply a single probabilistic jump operator right at the beginning of the free evolution time t2
# 2) we can remove the mixing time scale 
# 3)  wether we can remove the scalar couplings during t1 and t2 is subject to exploration, but firstly we assume that we need to consider them  

In [245]:
def GenH0_Ham_Part(offset,B0,zeeman_scalars,Jcoups,gamma):
    """
    Returns: the zeroth order Hamiltonian considered for dynamical evolution in nthe simulations, in OpenFermion format
    Args:
    offset: frequency offset for the spin Zeeman frequencies, in Hz
    B0: Strength of the magnetic field in Teslas
    zeeman_scalars: list of chemical shifts for spins, in ppm
    Jcoups: matrix of size N x N, N being the number of spins, that encodes the scalar couplings between spins (in Hz)
    gamma: gyromagnetic ratio of the spins (an homonuclear case is assumed) 
    """


    #offset = -46681
    #B0 = 9.3933
    w0 = -gamma*B0
    o1 = 2*np.pi*offset
    Nspins = len(zeeman_scalars)

    Hamiltonian0 = of.QubitOperator()

    for i in range(Nspins):
        w = o1+w0*zeeman_scalars[i]/1e6
        Hamiltonian0+=w*Sz(i)


    Interactions = []

    for i in range(Nspins):
        for j in range(i+1,Nspins):

            if Jcoups[i,j] != 0.0:

                Interactions.append(2*np.pi*Jcoups[i,j]*(Sx(i)*Sx(j)+Sy(i)*Sy(j)+Sz(i)*Sz(j)))
    
    return Hamiltonian0, Interactions

#####Rewriting, cleaning and tailoring some functions for QDrift simulation of the coherent part of the Hamiltonian
def BuildCohQDriftChann_fromOps(time_evol,Norm_ops,pks,Gamma):
    """
    It is assumed that all operators in Norm_ops corresponds to fragments of the coherent part of the Liouvillian 
    """
    Qdrift_chann = pks[0]*expm(-1j*Norm_ops[0]*time_evol*Gamma)
    for i in range(1,len(pks)):
        Qdrift_chann += pks[i]*expm(-1j*Norm_ops[i]*time_evol*Gamma)

    return Qdrift_chann

def CohQDriftEvol_fromNormOps(Norm_ops,pks,Gamma,time_evol,Nsteps,rho):
    
    rho_c = np.copy(rho)

    if time_evol==0:
        chann = np.eye(rho_c.shape[0])
    else:
        chann= BuildCohQDriftChann_fromOps(time_evol/Nsteps,Norm_ops,pks,Gamma)

    for i in range(Nsteps):
        rho_c = chann@rho_c

    return rho_c

def GenFID_SingJump_noTmix_GradField_SetMaxDisc(Ham,IntOps,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly,Lz,Nmax):
    """
    We aim to perform the simulation by setting a maximum number of samples from a QDrift channel to perform the simulation of t1 and t2 timescales
    to perform the simulation of NOESY spectrum. 
    Args:
    Ham: the Zeeman Hamiltonian 
    IntOps: the list of Hamiltonian fragments, such that their addition to the Zeeaman Hamiltonian define the coherent contribution to the Liovillian
    JumOps: List of jump operators to include in the simulation
    T1: total simulation time along the t1 axis
    T2: total simulation time along the t2 axis
    rho0: initial density matrix
    coil: the observable to measure 
    tmix: the mixing time
    dt1: the time step that defines the time-resolution of the grid along the t1 axis 
    dt2: the time step that defines the time resolution of the grid along the t2 axis
    Ntmix: the number of samples assumed to be taken during the mixing time through a QDroft channel. Note that in practice this channel is approximated as an effective one where
    the mixing time is removed
    Lx,Ly,Lz: collective angular momentum operators
    Nmax: maximum number of Qdrift samples for the simulation of t1,t2 times (i.e. the maximum number of samples is fixed to 2*Nmax for time propagation of t1+t2)
    """
    rho0 = rho0.flatten()

    Npts_grad = 100
    dim = rho0.shape[0]
    
    #R = sum(List_jumps)

    #In this version, we drop the dissipation channels of the evolution during T1 and T2
    #R = sum(JumpOps)

    #L_dt1 = expm((-1j*Ham+R)*dt1)

    #L_dt2 = expm((-1j*Ham+R)*dt2)

    Tpts1 = int(np.floor(T1/dt1))
    Tpts2 = int(np.floor(T2/dt2))

    print("Number of points for T1",Tpts1)
    print("Number of points for T2",Tpts2)
    #print("Number of Trotter steps for QDrift simulation of tmix", Ntmix)
    
    #pulse_mix = eff_QDriftChann_OneJump([Ham]+List_jumps,tmix,Ntmix)
    #pulse_mix = eff_QDriftChann_InitJump([Ham]+List_jumps,tmix,Ntmix)
    #QDriftChann([Ham]+List_jumps,tmix)
    #for i in range(Ntmix):
    #    pulse_mix = L_dtmix@pulse_mix

    #print("Difference between Trotterized pulse_mix and reference pulse_mix:", np.linalg.norm(pulse_mix-expm((-1j*Ham+R)*tmix)))

    pulse_90x = expm(-1j*Lx*np.pi/2)
    pulse_90y = expm(-1j*Ly*np.pi/2)
    pulse_90mx = expm(1j*Lx*np.pi/2)
    pulse_90my = expm(1j*Ly*np.pi/2)

    #First 90x pulse:
    rho_t = np.copy(rho0)
    rho_t = np.dot(pulse_90x,rho_t)

    rho_stack = []
    rho_stack.append(rho_t)

    ###Getting the list of operators and probabilities for the QDrift channel...
    Norm_ops, List_weights = Normalize_and_weightOps([Ham]+IntOps)

    List_weights = np.array(List_weights)
    Gamma =  np.sum(List_weights)

    pks = (1.0/Gamma)*List_weights

    #QDriftEvol_fromNormOps(Norm_ops,pks,Gamma,time_evol,Nsteps,rho)

    #rho_temp = np.copy(rho_t)
    for i in range(1,Tpts1):
        ####after the number of time points exceeds the maximum, we keep the number of samples in the QDrift channel fixed...
        if i > Nmax:
            #rho_t = QDriftEvol([Ham]+JumpOps,i*dt1,Nmax,rho0)
            rho_t = CohQDriftEvol_fromNormOps(Norm_ops,pks,Gamma,i*dt1,Nmax,rho0)

            #rho_temp = np.dot(L_dt1,rho_temp)
            rho_stack.append(rho_t)

        else:
            #rho_t = QDriftEvol([Ham]+JumpOps,i*dt1,i,rho0)
            rho_t = CohQDriftEvol_fromNormOps(Norm_ops,pks,Gamma,i*dt1,i,rho0)

            #rho_temp = np.dot(L_dt1,rho_temp)
            rho_stack.append(rho_t)


    rho_stack1_1 = []
    rho_stack1_2 = []
    rho_stack1_3 = []
    rho_stack1_4 = []

    #TODO: change this accordingly to include dissipation....
    #leftmix,rightmix = eff_QDriftChann_FinJump_Factorized([Ham]+List_jumps,tmix,Ntmix,pulse_90y)
    #pulse_mix = leftmix@rightmix

    #This is the last pulse that mimics the relaxation ocurring during the mixing time, thus eliminating the need of including mixing time

    ####TODO: 

    Hcoh = Ham+sum(IntOps)
    eff_pulse_90y = get_last_QDriftJump([Hcoh]+JumpOps,tmix,Ntmix,pulse_90y)
    

    ###Aplication of second 90 deg pulse

    print("Application of second 90 deg pulse")
    for i in range(Tpts1):
        rho_stack1_1.append(pulse_90x@rho_stack[i])
        rho_stack1_2.append(pulse_90y@rho_stack[i])
        rho_stack1_3.append(pulse_90mx@rho_stack[i])
        rho_stack1_4.append(pulse_90my@rho_stack[i])


    #####Aplication of gradient...
    rho_stack1_1 = FieldGrad(Npts_grad,rho_stack1_1,dim,Lz)
    rho_stack1_2 = FieldGrad(Npts_grad,rho_stack1_2,dim,Lz)
    rho_stack1_3 = FieldGrad(Npts_grad,rho_stack1_3,dim,Lz)
    rho_stack1_4 = FieldGrad(Npts_grad,rho_stack1_4,dim,Lz)

    #print("Mixing time")
    ###aplication of mixing
    #for i in range(Tpts1):
    #    rho_stack1_1.append(pulse_mix@rho_stack[i])
    #    rho_stack1_2.append(pulse_mix@rho_stack[i])
    #    rho_stack1_3.append(pulse_mix@rho_stack[i])
    #    rho_stack1_4.append(pulse_mix@rho_stack[i])

    #aplication of gradient...
    #rho_stack1_1 = FieldGrad(Npts_grad,rho_stack1_1,dim,Lz)
    #rho_stack1_2 = FieldGrad(Npts_grad,rho_stack1_2,dim,Lz)
    #rho_stack1_3 = FieldGrad(Npts_grad,rho_stack1_3,dim,Lz)
    #rho_stack1_4 = FieldGrad(Npts_grad,rho_stack1_4,dim,Lz)

    print("Application of third 90 deg pulse")
    #application of third pulse...
    for i in range(Tpts1):
        rho_stack1_1[i] = eff_pulse_90y@rho_stack1_1[i]
        rho_stack1_2[i] = eff_pulse_90y@rho_stack1_2[i]
        rho_stack1_3[i] = eff_pulse_90y@rho_stack1_3[i]
        rho_stack1_4[i] = eff_pulse_90y@rho_stack1_4[i]



    fid_temp_1 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_2 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_3 = np.zeros([Tpts2,Tpts1],dtype=complex)
    fid_temp_4 = np.zeros([Tpts2,Tpts1],dtype=complex)

    
    for i in range(Tpts1):
        rho1 = np.copy(rho_stack1_1[i])
        rho2 = np.copy(rho_stack1_2[i])
        rho3 = np.copy(rho_stack1_3[i])
        rho4 = np.copy(rho_stack1_4[i])

        rho1_t2 = np.copy(rho1)
        rho2_t2 = np.copy(rho2)
        rho3_t2 = np.copy(rho3)
        rho4_t2 = np.copy(rho4)

        for j in range(Tpts2):
            fid_temp_1[j,i] = np.dot(coil,rho1_t2)

            if j > Nmax:
                #rho1_t2 = QDriftEvol([Ham]+JumpOps,j*dt2,Nmax,rho1)
                rho1_t2 = CohQDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,Nmax,rho1)
            else:
                rho1_t2 = CohQDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,j,rho1)
            
            #rho1 = L_dt2@rho1

            fid_temp_2[j,i] = np.dot(coil,rho2_t2)
            if j > Nmax:
                rho2_t2 = CohQDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,Nmax,rho2)
            else:
                rho2_t2 = CohQDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,j,rho2)
            #rho2 = L_dt2@rho2

            fid_temp_3[j,i] = np.dot(coil,rho3_t2)
            if j > Nmax:
                rho3_t2 = CohQDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,Nmax,rho3)
            else:
                rho3_t2 = CohQDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,j,rho3)

            #rho3 = L_dt2@rho3

            fid_temp_4[j,i] = np.dot(coil,rho4_t2)
            if j > Nmax:
                rho4_t2 = CohQDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,Nmax,rho4)
            else:
                rho4_t2 = CohQDriftEvol_fromNormOps(Norm_ops,pks,Gamma,j*dt2,j,rho4)

            #rho4 = L_dt2@rho4
        print("Finishing iteration: ",i)
    
    return fid_temp_1, fid_temp_2, fid_temp_3, fid_temp_4





In [175]:
text4="""1      (0,0)   (0,0)   (0,0)   (0,0)   
  2      (0,0)   (0,0)   (0,0)   (1,1)   
  3      (0,0)   (0,0)   (0,0)   (1,0)   
  4      (0,0)   (0,0)   (0,0)   (1,-1)  
  5      (0,0)   (0,0)   (1,1)   (0,0)   
  6      (0,0)   (0,0)   (1,1)   (1,1)   
  7      (0,0)   (0,0)   (1,1)   (1,0)   
  8      (0,0)   (0,0)   (1,1)   (1,-1)  
  9      (0,0)   (0,0)   (1,0)   (0,0)   
  10     (0,0)   (0,0)   (1,0)   (1,1)   
  11     (0,0)   (0,0)   (1,0)   (1,0)   
  12     (0,0)   (0,0)   (1,0)   (1,-1)  
  13     (0,0)   (0,0)   (1,-1)  (0,0)   
  14     (0,0)   (0,0)   (1,-1)  (1,1)   
  15     (0,0)   (0,0)   (1,-1)  (1,0)   
  16     (0,0)   (0,0)   (1,-1)  (1,-1)  
  17     (0,0)   (1,1)   (0,0)   (0,0)   
  18     (0,0)   (1,1)   (0,0)   (1,1)   
  19     (0,0)   (1,1)   (0,0)   (1,0)   
  20     (0,0)   (1,1)   (0,0)   (1,-1)  
  21     (0,0)   (1,1)   (1,1)   (0,0)   
  22     (0,0)   (1,1)   (1,1)   (1,1)   
  23     (0,0)   (1,1)   (1,1)   (1,0)   
  24     (0,0)   (1,1)   (1,1)   (1,-1)  
  25     (0,0)   (1,1)   (1,0)   (0,0)   
  26     (0,0)   (1,1)   (1,0)   (1,1)   
  27     (0,0)   (1,1)   (1,0)   (1,0)   
  28     (0,0)   (1,1)   (1,0)   (1,-1)  
  29     (0,0)   (1,1)   (1,-1)  (0,0)   
  30     (0,0)   (1,1)   (1,-1)  (1,1)   
  31     (0,0)   (1,1)   (1,-1)  (1,0)   
  32     (0,0)   (1,1)   (1,-1)  (1,-1)  
  33     (0,0)   (1,0)   (0,0)   (0,0)   
  34     (0,0)   (1,0)   (0,0)   (1,1)   
  35     (0,0)   (1,0)   (0,0)   (1,0)   
  36     (0,0)   (1,0)   (0,0)   (1,-1)  
  37     (0,0)   (1,0)   (1,1)   (0,0)   
  38     (0,0)   (1,0)   (1,1)   (1,1)   
  39     (0,0)   (1,0)   (1,1)   (1,0)   
  40     (0,0)   (1,0)   (1,1)   (1,-1)  
  41     (0,0)   (1,0)   (1,0)   (0,0)   
  42     (0,0)   (1,0)   (1,0)   (1,1)   
  43     (0,0)   (1,0)   (1,0)   (1,0)   
  44     (0,0)   (1,0)   (1,0)   (1,-1)  
  45     (0,0)   (1,0)   (1,-1)  (0,0)   
  46     (0,0)   (1,0)   (1,-1)  (1,1)   
  47     (0,0)   (1,0)   (1,-1)  (1,0)   
  48     (0,0)   (1,0)   (1,-1)  (1,-1)  
  49     (0,0)   (1,-1)  (0,0)   (0,0)   
  50     (0,0)   (1,-1)  (0,0)   (1,1)   
  51     (0,0)   (1,-1)  (0,0)   (1,0)   
  52     (0,0)   (1,-1)  (0,0)   (1,-1)  
  53     (0,0)   (1,-1)  (1,1)   (0,0)   
  54     (0,0)   (1,-1)  (1,1)   (1,1)   
  55     (0,0)   (1,-1)  (1,1)   (1,0)   
  56     (0,0)   (1,-1)  (1,1)   (1,-1)  
  57     (0,0)   (1,-1)  (1,0)   (0,0)   
  58     (0,0)   (1,-1)  (1,0)   (1,1)   
  59     (0,0)   (1,-1)  (1,0)   (1,0)   
  60     (0,0)   (1,-1)  (1,0)   (1,-1)  
  61     (0,0)   (1,-1)  (1,-1)  (0,0)   
  62     (0,0)   (1,-1)  (1,-1)  (1,1)   
  63     (0,0)   (1,-1)  (1,-1)  (1,0)   
  64     (0,0)   (1,-1)  (1,-1)  (1,-1)  
  65     (1,1)   (0,0)   (0,0)   (0,0)   
  66     (1,1)   (0,0)   (0,0)   (1,1)   
  67     (1,1)   (0,0)   (0,0)   (1,0)   
  68     (1,1)   (0,0)   (0,0)   (1,-1)  
  69     (1,1)   (0,0)   (1,1)   (0,0)   
  70     (1,1)   (0,0)   (1,1)   (1,1)   
  71     (1,1)   (0,0)   (1,1)   (1,0)   
  72     (1,1)   (0,0)   (1,1)   (1,-1)  
  73     (1,1)   (0,0)   (1,0)   (0,0)   
  74     (1,1)   (0,0)   (1,0)   (1,1)   
  75     (1,1)   (0,0)   (1,0)   (1,0)   
  76     (1,1)   (0,0)   (1,0)   (1,-1)  
  77     (1,1)   (0,0)   (1,-1)  (0,0)   
  78     (1,1)   (0,0)   (1,-1)  (1,1)   
  79     (1,1)   (0,0)   (1,-1)  (1,0)   
  80     (1,1)   (0,0)   (1,-1)  (1,-1)  
  81     (1,1)   (1,1)   (0,0)   (0,0)   
  82     (1,1)   (1,1)   (0,0)   (1,1)   
  83     (1,1)   (1,1)   (0,0)   (1,0)   
  84     (1,1)   (1,1)   (0,0)   (1,-1)  
  85     (1,1)   (1,1)   (1,1)   (0,0)   
  86     (1,1)   (1,1)   (1,1)   (1,1)   
  87     (1,1)   (1,1)   (1,1)   (1,0)   
  88     (1,1)   (1,1)   (1,1)   (1,-1)  
  89     (1,1)   (1,1)   (1,0)   (0,0)   
  90     (1,1)   (1,1)   (1,0)   (1,1)   
  91     (1,1)   (1,1)   (1,0)   (1,0)   
  92     (1,1)   (1,1)   (1,0)   (1,-1)  
  93     (1,1)   (1,1)   (1,-1)  (0,0)   
  94     (1,1)   (1,1)   (1,-1)  (1,1)   
  95     (1,1)   (1,1)   (1,-1)  (1,0)   
  96     (1,1)   (1,1)   (1,-1)  (1,-1)  
  97     (1,1)   (1,0)   (0,0)   (0,0)   
  98     (1,1)   (1,0)   (0,0)   (1,1)   
  99     (1,1)   (1,0)   (0,0)   (1,0)   
  100    (1,1)   (1,0)   (0,0)   (1,-1)  
  101    (1,1)   (1,0)   (1,1)   (0,0)   
  102    (1,1)   (1,0)   (1,1)   (1,1)   
  103    (1,1)   (1,0)   (1,1)   (1,0)   
  104    (1,1)   (1,0)   (1,1)   (1,-1)  
  105    (1,1)   (1,0)   (1,0)   (0,0)   
  106    (1,1)   (1,0)   (1,0)   (1,1)   
  107    (1,1)   (1,0)   (1,0)   (1,0)   
  108    (1,1)   (1,0)   (1,0)   (1,-1)  
  109    (1,1)   (1,0)   (1,-1)  (0,0)   
  110    (1,1)   (1,0)   (1,-1)  (1,1)   
  111    (1,1)   (1,0)   (1,-1)  (1,0)   
  112    (1,1)   (1,0)   (1,-1)  (1,-1)  
  113    (1,1)   (1,-1)  (0,0)   (0,0)   
  114    (1,1)   (1,-1)  (0,0)   (1,1)   
  115    (1,1)   (1,-1)  (0,0)   (1,0)   
  116    (1,1)   (1,-1)  (0,0)   (1,-1)  
  117    (1,1)   (1,-1)  (1,1)   (0,0)   
  118    (1,1)   (1,-1)  (1,1)   (1,1)   
  119    (1,1)   (1,-1)  (1,1)   (1,0)   
  120    (1,1)   (1,-1)  (1,1)   (1,-1)  
  121    (1,1)   (1,-1)  (1,0)   (0,0)   
  122    (1,1)   (1,-1)  (1,0)   (1,1)   
  123    (1,1)   (1,-1)  (1,0)   (1,0)   
  124    (1,1)   (1,-1)  (1,0)   (1,-1)  
  125    (1,1)   (1,-1)  (1,-1)  (0,0)   
  126    (1,1)   (1,-1)  (1,-1)  (1,1)   
  127    (1,1)   (1,-1)  (1,-1)  (1,0)   
  128    (1,1)   (1,-1)  (1,-1)  (1,-1)  
  129    (1,0)   (0,0)   (0,0)   (0,0)   
  130    (1,0)   (0,0)   (0,0)   (1,1)   
  131    (1,0)   (0,0)   (0,0)   (1,0)   
  132    (1,0)   (0,0)   (0,0)   (1,-1)  
  133    (1,0)   (0,0)   (1,1)   (0,0)   
  134    (1,0)   (0,0)   (1,1)   (1,1)   
  135    (1,0)   (0,0)   (1,1)   (1,0)   
  136    (1,0)   (0,0)   (1,1)   (1,-1)  
  137    (1,0)   (0,0)   (1,0)   (0,0)   
  138    (1,0)   (0,0)   (1,0)   (1,1)   
  139    (1,0)   (0,0)   (1,0)   (1,0)   
  140    (1,0)   (0,0)   (1,0)   (1,-1)  
  141    (1,0)   (0,0)   (1,-1)  (0,0)   
  142    (1,0)   (0,0)   (1,-1)  (1,1)   
  143    (1,0)   (0,0)   (1,-1)  (1,0)   
  144    (1,0)   (0,0)   (1,-1)  (1,-1)  
  145    (1,0)   (1,1)   (0,0)   (0,0)   
  146    (1,0)   (1,1)   (0,0)   (1,1)   
  147    (1,0)   (1,1)   (0,0)   (1,0)   
  148    (1,0)   (1,1)   (0,0)   (1,-1)  
  149    (1,0)   (1,1)   (1,1)   (0,0)   
  150    (1,0)   (1,1)   (1,1)   (1,1)   
  151    (1,0)   (1,1)   (1,1)   (1,0)   
  152    (1,0)   (1,1)   (1,1)   (1,-1)  
  153    (1,0)   (1,1)   (1,0)   (0,0)   
  154    (1,0)   (1,1)   (1,0)   (1,1)   
  155    (1,0)   (1,1)   (1,0)   (1,0)   
  156    (1,0)   (1,1)   (1,0)   (1,-1)  
  157    (1,0)   (1,1)   (1,-1)  (0,0)   
  158    (1,0)   (1,1)   (1,-1)  (1,1)   
  159    (1,0)   (1,1)   (1,-1)  (1,0)   
  160    (1,0)   (1,1)   (1,-1)  (1,-1)  
  161    (1,0)   (1,0)   (0,0)   (0,0)   
  162    (1,0)   (1,0)   (0,0)   (1,1)   
  163    (1,0)   (1,0)   (0,0)   (1,0)   
  164    (1,0)   (1,0)   (0,0)   (1,-1)  
  165    (1,0)   (1,0)   (1,1)   (0,0)   
  166    (1,0)   (1,0)   (1,1)   (1,1)   
  167    (1,0)   (1,0)   (1,1)   (1,0)   
  168    (1,0)   (1,0)   (1,1)   (1,-1)  
  169    (1,0)   (1,0)   (1,0)   (0,0)   
  170    (1,0)   (1,0)   (1,0)   (1,1)   
  171    (1,0)   (1,0)   (1,0)   (1,0)   
  172    (1,0)   (1,0)   (1,0)   (1,-1)  
  173    (1,0)   (1,0)   (1,-1)  (0,0)   
  174    (1,0)   (1,0)   (1,-1)  (1,1)   
  175    (1,0)   (1,0)   (1,-1)  (1,0)   
  176    (1,0)   (1,0)   (1,-1)  (1,-1)  
  177    (1,0)   (1,-1)  (0,0)   (0,0)   
  178    (1,0)   (1,-1)  (0,0)   (1,1)   
  179    (1,0)   (1,-1)  (0,0)   (1,0)   
  180    (1,0)   (1,-1)  (0,0)   (1,-1)  
  181    (1,0)   (1,-1)  (1,1)   (0,0)   
  182    (1,0)   (1,-1)  (1,1)   (1,1)   
  183    (1,0)   (1,-1)  (1,1)   (1,0)   
  184    (1,0)   (1,-1)  (1,1)   (1,-1)  
  185    (1,0)   (1,-1)  (1,0)   (0,0)   
  186    (1,0)   (1,-1)  (1,0)   (1,1)   
  187    (1,0)   (1,-1)  (1,0)   (1,0)   
  188    (1,0)   (1,-1)  (1,0)   (1,-1)  
  189    (1,0)   (1,-1)  (1,-1)  (0,0)   
  190    (1,0)   (1,-1)  (1,-1)  (1,1)   
  191    (1,0)   (1,-1)  (1,-1)  (1,0)   
  192    (1,0)   (1,-1)  (1,-1)  (1,-1)  
  193    (1,-1)  (0,0)   (0,0)   (0,0)   
  194    (1,-1)  (0,0)   (0,0)   (1,1)   
  195    (1,-1)  (0,0)   (0,0)   (1,0)   
  196    (1,-1)  (0,0)   (0,0)   (1,-1)  
  197    (1,-1)  (0,0)   (1,1)   (0,0)   
  198    (1,-1)  (0,0)   (1,1)   (1,1)   
  199    (1,-1)  (0,0)   (1,1)   (1,0)   
  200    (1,-1)  (0,0)   (1,1)   (1,-1)  
  201    (1,-1)  (0,0)   (1,0)   (0,0)   
  202    (1,-1)  (0,0)   (1,0)   (1,1)   
  203    (1,-1)  (0,0)   (1,0)   (1,0)   
  204    (1,-1)  (0,0)   (1,0)   (1,-1)  
  205    (1,-1)  (0,0)   (1,-1)  (0,0)   
  206    (1,-1)  (0,0)   (1,-1)  (1,1)   
  207    (1,-1)  (0,0)   (1,-1)  (1,0)   
  208    (1,-1)  (0,0)   (1,-1)  (1,-1)  
  209    (1,-1)  (1,1)   (0,0)   (0,0)   
  210    (1,-1)  (1,1)   (0,0)   (1,1)   
  211    (1,-1)  (1,1)   (0,0)   (1,0)   
  212    (1,-1)  (1,1)   (0,0)   (1,-1)  
  213    (1,-1)  (1,1)   (1,1)   (0,0)   
  214    (1,-1)  (1,1)   (1,1)   (1,1)   
  215    (1,-1)  (1,1)   (1,1)   (1,0)   
  216    (1,-1)  (1,1)   (1,1)   (1,-1)  
  217    (1,-1)  (1,1)   (1,0)   (0,0)   
  218    (1,-1)  (1,1)   (1,0)   (1,1)   
  219    (1,-1)  (1,1)   (1,0)   (1,0)   
  220    (1,-1)  (1,1)   (1,0)   (1,-1)  
  221    (1,-1)  (1,1)   (1,-1)  (0,0)   
  222    (1,-1)  (1,1)   (1,-1)  (1,1)   
  223    (1,-1)  (1,1)   (1,-1)  (1,0)   
  224    (1,-1)  (1,1)   (1,-1)  (1,-1)  
  225    (1,-1)  (1,0)   (0,0)   (0,0)   
  226    (1,-1)  (1,0)   (0,0)   (1,1)   
  227    (1,-1)  (1,0)   (0,0)   (1,0)   
  228    (1,-1)  (1,0)   (0,0)   (1,-1)  
  229    (1,-1)  (1,0)   (1,1)   (0,0)   
  230    (1,-1)  (1,0)   (1,1)   (1,1)   
  231    (1,-1)  (1,0)   (1,1)   (1,0)   
  232    (1,-1)  (1,0)   (1,1)   (1,-1)  
  233    (1,-1)  (1,0)   (1,0)   (0,0)   
  234    (1,-1)  (1,0)   (1,0)   (1,1)   
  235    (1,-1)  (1,0)   (1,0)   (1,0)   
  236    (1,-1)  (1,0)   (1,0)   (1,-1)  
  237    (1,-1)  (1,0)   (1,-1)  (0,0)   
  238    (1,-1)  (1,0)   (1,-1)  (1,1)   
  239    (1,-1)  (1,0)   (1,-1)  (1,0)   
  240    (1,-1)  (1,0)   (1,-1)  (1,-1)  
  241    (1,-1)  (1,-1)  (0,0)   (0,0)   
  242    (1,-1)  (1,-1)  (0,0)   (1,1)   
  243    (1,-1)  (1,-1)  (0,0)   (1,0)   
  244    (1,-1)  (1,-1)  (0,0)   (1,-1)  
  245    (1,-1)  (1,-1)  (1,1)   (0,0)   
  246    (1,-1)  (1,-1)  (1,1)   (1,1)   
  247    (1,-1)  (1,-1)  (1,1)   (1,0)   
  248    (1,-1)  (1,-1)  (1,1)   (1,-1)  
  249    (1,-1)  (1,-1)  (1,0)   (0,0)   
  250    (1,-1)  (1,-1)  (1,0)   (1,1)   
  251    (1,-1)  (1,-1)  (1,0)   (1,0)   
  252    (1,-1)  (1,-1)  (1,0)   (1,-1)  
  253    (1,-1)  (1,-1)  (1,-1)  (0,0)   
  254    (1,-1)  (1,-1)  (1,-1)  (1,1)   
  255    (1,-1)  (1,-1)  (1,-1)  (1,0)   
  256    (1,-1)  (1,-1)  (1,-1)  (1,-1)"""

data_ala = read_spinach_info(text4)

basis_ala = build_list_ISTs(data_ala)
prefacts,Symb_ALA_basis = build_symbolic_list_ISTs(data_ala)

#Normbasis = NormalizeBasis(basis,n_qubits=4,checkOrth=True) I have verified the orthonormalization of the basis
Normbasis_ala = NormalizeBasis(basis_ala,n_qubits=4,checkOrth=False)
Normbasis_ala = np.array(Normbasis_ala)


In [201]:
###QDrift for the Hamiltonian simulation of the coherent part sounds like a simplified aapproach to this...

#First we need to generate the list of operators to sample over in the QDrift channel, we need to generate the jump operators...

from analytical_fit import Get_Det_And_Rates, get_chemical_shifts
from basis_utils import  MatRepLib
from simulation_utils import GenH0_Ham

B0 = 18.78
offset = 3759.37
freq1 = -799607997.1075
freq2 = -799606158.0159
freq3 = -799606158.0159
freq4 = -799606158.0159

freqs = np.array([freq1,freq2,freq3,freq4])
tc = 18.8e-12
coords = np.array([
    [0.6861   ,0.2705   ,1.5010],
    [1.3077    ,1.1298   ,-1.3993],
    [0.7905    ,2.2125   ,-0.0860],
    [2.3693    ,1.3798    ,0.0233]
]
)
coords =coords*1e-10
Nspins =4
gammaH = 2.6752e+08

Jcoups = np.zeros([Nspins,Nspins])

Jcoups[0,1] = 7.0
Jcoups[0,2] = 7.0
Jcoups[0,3] = 7.0


####Getting the corresponding jump operators for the set of parameters chosen for the simulation of the spectrum

zeeman_scalar_1 = 3.69
zeeman_scalar_2 = 1.39
zeeman_scalar_3 = 1.39
zeeman_scalar_4 = 1.39

zeeman_scalars = [zeeman_scalar_1,zeeman_scalar_2,zeeman_scalar_3,zeeman_scalar_4]

chem_shifts = get_chemical_shifts(gammaH,B0,zeeman_scalars)

#list_jumps, list_damp_rates, list_dets=Get_Det_And_Rates(2*np.pi*freqs,tc,coords,Nspins,gammaH,chem_shifts)
#list_jumps, list_symb_rates, list_damp_rates, list_dets = Get_Det_And_Rates_latex(2*np.pi*freqs,tc,coords,Nspins,gammaH,chem_shifts)

list_jumps, list_damp_rates, list_dets = Get_Det_And_Rates(freqs,tc,coords,Nspins,gammaH,chem_shifts)

###We verified, as expected that the dominant relaxation channels correspond to ZZ relaxation channels Z0Z1 Z0Z2 Z1Z2

#We build their corresponding matrix representation in the Pauli basis...
#ListJumps = []



#ListJumps.append(1.0813667*MatRepLib(Normbasis_ala,Sz(1)*Sz(2),Sz(1)*Sz(2),n_qubits=4))
#ListJumps.append(1.047784*MatRepLib(Normbasis_ala,Sz(2)*Sz(3),Sz(2)*Sz(3),n_qubits=4))
#ListJumps.append(1.0328497*MatRepLib(Normbasis_ala,Sz(1)*Sz(3),Sz(1)*Sz(3),n_qubits=4))




In [221]:
#We build the Hamiltonian fragments that comprise the coherent Hamiltonian from Spinach-computed matrices...

loadMat = spio.loadmat('./data/NOESYdata_ALA_experiment_miscHJs.mat',squeeze_me=True)


H_ala_coh_tot = loadMat['p']['H'].item().toarray()
H_ala_noJs = loadMat['p']['H_noJs'].item().toarray()
H_ala_J12 = loadMat['p']['H_J12'].item().toarray()
H_ala_J13 = loadMat['p']['H_J13'].item().toarray()
H_ala_J14 = loadMat['p']['H_J14'].item().toarray()

Zeem_Ham = H_ala_noJs

S1S2 = H_ala_J12 - Zeem_Ham
S1S3 = H_ala_J13 - Zeem_Ham
S1S4 = H_ala_J14 - Zeem_Ham

ListInts = [S1S2,S1S3,S1S4]



In [224]:
Nmax = 10
#fid1_effJN10, fid2_effJN10, fid3_effJN10, fid4_effJN10  = GenFID_SingJump_noTmix_GradField_SetMaxDisc(Zeem_Ham,ListInts,JumpOps,T1,T2,rho0,coil,tmix,dt1,dt2,Ntmix,Lx,Ly,Lz,Nmax)

In [223]:
####Saving everything in a file for later retrieval...
ALA_exp_params = {}
ALA_exp_params['ZeemanH'] = Zeem_Ham
ALA_exp_params['ListInts'] =  ListInts
ALA_exp_params['JumpOps'] = JumpOps
ALA_exp_params['T1'] = T1
ALA_exp_params['T2'] = T2
ALA_exp_params['rho0'] = rho0
ALA_exp_params['coil'] = coil
ALA_exp_params['tmix'] = tmix
ALA_exp_params['dt1'] = dt1
ALA_exp_params['dt2'] = dt2
ALA_exp_params['Lx'] = Lx
ALA_exp_params['Ly'] = Ly
ALA_exp_params['Lz'] = Lz

#ALA_exp_params['dt2'] = dt2
with open('./data/ALA_expGradField_AllParams.pk', 'wb') as handle:
    pickle.dump(ALA_exp_params, handle)



In [278]:
Norm_ops, List_weights = Normalize_and_weightOps([Zeem_Ham] + ListInts)
List_weights = np.array(List_weights)
Gamma = np.sum(List_weights)
pks = (1.0 / Gamma) * List_weights

L_dt1 = BuildCohQDriftChann_fromOps(dt1,Norm_ops,pks,Gamma)

In [280]:
rho_test = pulse_90x@np.copy(rho0)
rho_ref = pulse_90x@np.copy(rho0)

for i in range(1024):
    rho_test = L_dt1@rho_test
    rho_ref = expm(-1j*H_ala_coh_tot*dt1)@rho_ref


In [282]:
np.linalg.norm(rho_test - rho_ref)/np.linalg.norm(rho_ref)

0.00013415813273991127

In [252]:
np.linalg.norm(L_dt1)

15.967910899005311

In [251]:
np.linalg.norm(H_ala_coh_tot-Zeem_Ham-sum(ListInts))

5.291167361697463e-10

In [233]:
T1/512

0.0006252

In [234]:
dt1

0.0003126

In [241]:
###Load data 
f= open('./data/ALA_SimplifiedFIDParallel10maxLayers.pk','rb')
dat =pickle.load(f)

fid1_N10 = dat['fid1']
fid2_N10 = dat['fid2']
fid3_N10 = dat['fid3']
fid4_N10 = dat['fid4']


In [242]:
cos_fid = fid1_N10 - fid3_N10
sin_fid = fid2_N10 - fid4_N10

savemat('ALA_truncR_FIDcos_effJGrad_N10.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_effJGrad_N10.mat', {'FID_sin': sin_fid})

In [239]:
f= open('./data/ALA_SimplifiedFIDParallel_noQDrift.pk','rb')
dat =pickle.load(f)

fid1 = dat['fid1']
fid2 = dat['fid2']
fid3 = dat['fid3']
fid4 = dat['fid4']

cos_fid = fid1 - fid3
sin_fid = fid2 - fid4

savemat('ALA_truncR_FIDcos_effJGrad_noQdrift.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_effJGrad_noQDrift.mat', {'FID_sin': sin_fid})


In [240]:
cos_fid.shape

(1024, 1024)

In [288]:
f= open('./data/ALA_SimplifiedFIDParallelmaxNQD.pk','rb')
dat =pickle.load(f)

fid1 = dat['fid1']
fid2 = dat['fid2']
fid3 = dat['fid3']
fid4 = dat['fid4']

cos_fid = fid1 - fid3
sin_fid = fid2 - fid4

savemat('ALA_truncR_FIDcos_effJGrad_maxQD.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_effJGrad_maxQD.mat', {'FID_sin': sin_fid})


In [284]:
cos_fid.shape

(10240, 10240)

In [290]:

f= open('./data/ALA_SimplifiedFIDParallel_noQDrift_noJs.pk','rb')
dat =pickle.load(f)

fid1 = dat['fid1']
fid2 = dat['fid2']
fid3 = dat['fid3']
fid4 = dat['fid4']

cos_fid = fid1 - fid3
sin_fid = fid2 - fid4

savemat('ALA_truncR_FIDcos_effJGrad_noQD_noJs.mat', {'FID_cos': cos_fid})
savemat('ALA_truncR_FIDsin_effJGrad_noQD_noJs.mat', {'FID_sin': sin_fid})

In [287]:
####

(1024, 1024)